In [1]:
FileUtils.cd '../../'
require './config/boot'
APP_PATH  = File.expand_path('config/application') unless defined?(APP_PATH)
require APP_PATH
Rails.application.require_environment!

true

In [2]:
require "#{Rails.root}/app/models/subject"
require "#{Rails.root}/app/models/year_datum"
require "#{Rails.root}/app/models/faculty"
require "#{Rails.root}/app/models/subject_score"
require "#{Rails.root}/app/models/summarized_subject"

true

In [3]:
require 'open-uri'
require 'nokogiri'
require 'mechanize'
require 'robotex'

false

## 設定

In [23]:
url = 'https://syllabus.doshisha.ac.jp/html/2015/37/137584004.html'

"https://syllabus.doshisha.ac.jp/html/2015/37/137584004.html"

In [24]:
agent = Mechanize.new
robotex = Robotex.new
p "robots: #{robotex.allowed?(url)}"
page = agent.get(url)
''

"robots: false"


""

## スクレイピング
タグ（項目？）ごとにtable, textのハッシュを作成

In [25]:
data = page.css('body > div > table:nth-child(2) > tbody > tr:nth-child(4) > td')
children=data.children
''

""

In [26]:
table_hash = {}
text_hash = {}
tag = 'init'

"init"

In [27]:
children.each do |t|
  text = t.text.strip
  if t.node_name == 'table'
    table_hash[tag] = t
  elsif text.present?
    if ret = text.match(/^＜(.+)＞$/)
      p "tag #{tag = ret[1]}"
    else
      text_hash[tag] = text
    end
  end
end
''
table_hash.keys

"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag 備考"


["授業計画", "成績評価基準"]

In [35]:
table_hash["成績評価基準"].css('tbody > tr > td:nth-child(1)').text

"\n平常点(出席，クラス参加，グループ作業の成果等) \n"

In [36]:
table_hash["成績評価基準"].css('tbody > tr > td:nth-child(2)').text

"\n100% \n"

In [29]:
text_hash

{"概要"=>"この授業は，政治学科の1年次生を対象に春学期に開講される「政治学入門」の続編的な科目として位置付けられています。具体的には，「政治学入門」で学んだアカデミック・リテラシーを踏まえた上で，レジュメ作成・報告・レポート作成の技術をさらに向上させていくことを目的としています。実際，大学での「学問」においては，様々な知的作業を「一人」で行っていかなければならない場面が多々あります。そうして自らが発表したものは，今度は自分以外の多くの人から様々に評価されていくものです。　『論語』の有名な一節に「学びて思わざれば則ち罔し（くらし），思いて学ばざれば則ち殆し（あやうし）」とあるように，自ら考えることと他者から学ぶことは，ともに車の両輪のごとく大切な学問的態度です。受講生には，この授業を学問成果のアウトプットと，学問的批判のインプットに関するトレーニングの場として活用してほしいと思います。", "到達目標"=>"政治学を学ぶ上での基礎的な能力の向上", "成績評価基準"=>"①出席，②演習への取り組み，③課題の提出及びその内容をもとにして評価を行います。", "備考"=>"前期の講義内容を復習しておくこと。 "}

## テキストから書籍名を取ってくる

In [30]:
t=table_hash["参考文献"].text

NoMethodError: undefined method `text' for nil:NilClass

In [13]:
t.match(/『(.+)』/)[1]

"本当は恐いC言語"

In [14]:
from_syllabus = '978-4798032627'

"978-4798032627"

In [15]:
from_amazon = '978-4798032627'

"978-4798032627"

In [16]:
from_syllabus == from_amazon

true

SyntaxError: unexpected tIDENTIFIER, expecting end-of-input


In [20]:
"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=deep+learning&rh=n:465392,k:deep+learning&linkCode=ll2&tag=johshisha-22&linkId=7872c005eaf69e724d88d0c4701beedb"

"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=deep+learning&rh=n:465392,k:deep+learning&linkCode=ll2&tag=johshisha-22&linkId=7872c005eaf69e724d88d0c4701beedb"

In [21]:
"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=aps&field-keywords=%E6%9C%AC%E5%BD%93%E3%81%AF%E6%80%96%E3%81%84C%E8%A8%80%E8%AA%9E&rh=i:aps,k:%E6%9C%AC%E5%BD%93%E3%81%AF%E6%80%96%E3%81%84C%E8%A8%80%E8%AA%9E&linkCode=ll2&tag=johshisha-22&linkId=f538bec0edfb9ccee1dbc35725f5cf24"

"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=aps&field-keywords=%E6%9C%AC%E5%BD%93%E3%81%AF%E6%80%96%E3%81%84C%E8%A8%80%E8%AA%9E&rh=i:aps,k:%E6%9C%AC%E5%BD%93%E3%81%AF%E6%80%96%E3%81%84C%E8%A8%80%E8%AA%9E&linkCode=ll2&tag=johshisha-22&linkId=f538bec0edfb9ccee1dbc35725f5cf24"

In [22]:
"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=%E6%9C%AC%E5%BD%93%E3%81%AF%E6%80%96%E3%81%84C%E8%A8%80%E8%AA%9E&rh=n:465392,k:%E6%9C%AC%E5%BD%93%E3%81%AF%E6%80%96%E3%81%84C%E8%A8%80%E8%AA%9E&linkCode=ll2&tag=johshisha-22&linkId=9ce2044061980e18e3bb5d615eb52f84"

"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=%E6%9C%AC%E5%BD%93%E3%81%AF%E6%80%96%E3%81%84C%E8%A8%80%E8%AA%9E&rh=n:465392,k:%E6%9C%AC%E5%BD%93%E3%81%AF%E6%80%96%E3%81%84C%E8%A8%80%E8%AA%9E&linkCode=ll2&tag=johshisha-22&linkId=9ce2044061980e18e3bb5d615eb52f84"

# Amazon URL
root: "https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=#{keyword}&rh=n:465392,k:#{keyword}&linkCode=ll2&tag=johshisha-22"

In [1]:
keyword = "ゼロから始める"
url = "https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=#{keyword}&rh=n:465392,k:#{keyword}&linkCode=ll2&tag=johshisha-22"

"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=ゼロから始める&rh=n:465392,k:ゼロから始める&linkCode=ll2&tag=johshisha-22"